In [ ]:
from graphql.utils.schema_printer import print_schema
from graphql_compiler.macros import (
    register_macro_edge, get_schema_with_macros, perform_macro_expansion
)

from demo.server.demo_helpers import (
    get_schema, execute_query, pretty_print_data, get_empty_macro_registry, set_verbose_mode
)

In [ ]:
macro_registry = get_empty_macro_registry()

## Macro edges
Edges that are "real" in the schema, but are actually defined using GraphQL syntax and don't really exist in the underlying database(s).

They are especially valuable for:
- hiding database normalization
- ensuring that different services use consistent definitions and methodologies
- defining domain-specific functionality for a specific product

This is how you define a new macro edge, called `out_Region_RegisteredAirlines` between `Region` and `Airline`.
It accounts for the fact that airlines are registered in *countries*, and not *geographic regions*, then hides that from the user.

In [ ]:
macro_edge_graphql = '''{
    Region @macro_edge_definition(name: "out_Region_RegisteredAirlines") {
        out_GeographicArea_SubArea @recurse(depth: 5) {
            ... on Country {
                in_Airline_RegisteredIn @macro_edge_target {
                    id
                }
            }
        }
    }
}'''
macro_edge_args = {}
register_macro_edge(macro_registry, macro_edge_graphql, macro_edge_args)

Similarly, this is how we define a new macro that allows us to easily look up all airports in a region.

In [ ]:
macro_edge_graphql = '''{
    Region @macro_edge_definition(name: "out_Region_ContainedAirports") {
        out_GeographicArea_SubArea @recurse(depth: 5) {
            ... on Country {
                in_Airport_BasedIn @macro_edge_target {
                    id
                }
            }
        }
    }
}'''
macro_edge_args = {}
register_macro_edge(macro_registry, macro_edge_graphql, macro_edge_args)

Defined macros become part of the schema, and are invisible to the querying user.

In [ ]:
schema_with_macros = get_schema_with_macros(macro_registry)
print(print_schema(schema_with_macros))

When a query uses a macro edge, the macro is expanded before execution – producing a valid, macro-less query.

In [ ]:
# European airlines with names starting with "W" and ending in "Air"
query = '''{
    Region {
        name @filter(op_name: "=", value: ["$region"])
        
        out_Region_RegisteredAirlines {
            name @filter(op_name: "ends_with", value: ["$suffix"])
                 @filter(op_name: "starts_with", value: ["$prefix"])
                 @output(out_name: "airline")
        }
    }
}'''
args = {
    'region': 'Europe',
    'prefix': 'W',
    'suffix': 'Air',
}
expanded_query, expanded_args = perform_macro_expansion(macro_registry, query, args)
print(expanded_query)
print(expanded_args)
_, result = execute_query(expanded_query, expanded_args)
pretty_print_data(result)

### Macros with arguments

Macro edges can take arguments as well. Consider the below macro, that connects `Airport` vertices to all other `Airport` vertices which can be reached by taking only direct flights operated by European carriers.

In [ ]:
macro_edge_graphql = '''{
    Airport @macro_edge_definition(name: "out_DirectlyConnectedViaEuropeanAirline_ToAirport") {
        in_FlightRoute_FromAirport {
            stops @filter(op_name: "=", value: ["$stops"])
        
            out_FlightRoute_OperatingAirline {
                out_Airline_RegisteredIn {
                    in_GeographicArea_SubArea @recurse(depth: 5) {
                        name @filter(op_name: "=", value: ["$region_name"])
                    }
                }
            }
            out_FlightRoute_ToAirport @macro_edge_target {
                id
            }
        }
    }
}'''
macro_edge_args = {
    'region_name': 'Europe',
    'stops': 0,
}
register_macro_edge(macro_registry, macro_edge_graphql, macro_edge_args)

In [ ]:
# Direct flights operated by European airlines starting in the Caribbean
query = '''{
    Region {
        name @filter(op_name: "=", value: ["$region"])
        
        out_Region_ContainedAirports {
            name @output(out_name: "airport_name")
            city_served @output(out_name: "airport_city")
            iata_code @output(out_name: "airport_code")
            
            out_DirectlyConnectedViaEuropeanAirline_ToAirport {
                name @output(out_name: "destination_name")
                city_served @output(out_name: "destination_city")
                iata_code @output(out_name: "destination_code")
            }
        }
    }
}'''
args = {
    'region': 'Caribbean',
}
expanded_query, expanded_args = perform_macro_expansion(macro_registry, query, args)
print(expanded_query)
print(expanded_args)
_, result = execute_query(expanded_query, expanded_args)
pretty_print_data(result)

# The repository title promised cross-database querying. So where is it?

**We've been doing it all along!** The countries and regions data is in a graph database called OrientDB, and the airlines, airports, and flights information is in Postgres.

You probably have noticed that `execute_query` always returned two arguments, and we always ignored the first one. That's the distributed query plan!

In [ ]:
from graphql_compiler.schema_transformation.query_plan import print_query_plan

In [ ]:
query_plan, result = execute_query(expanded_query, expanded_args)
print(print_query_plan(query_plan))

In [ ]:
set_verbose_mode(True)
query_plan, result = execute_query(expanded_query, expanded_args)
set_verbose_mode(False)

### Merging schemas

The overall process is the following:
- Get multiple schemas you'd like to merge together, and assign them unique identifiers.
- Define any edges that cross from one schema to another, as the below cell shows.
- If any schemas happen to define the same type names, use the GraphQL compiler's type renaming functionality to resolve the conflict.
- Use the `merge_schemas()` function call to generate a merged schema descriptor, which includes a unified GraphQL schema and additional metadata used by the compiler.

For full details, look at `cross_db_query.py`. 

In [ ]:
from graphql_compiler.schema_transformation.merge_schemas import (
    CrossSchemaEdgeDescriptor, FieldReference,
)
cross_schema_edges = [
    CrossSchemaEdgeDescriptor(
        edge_name='Airport_BasedIn',
        outbound_field_reference=FieldReference(
            schema_id='postgres',
            type_name='Airport',
            field_name='alpha2_country',
        ),
        inbound_field_reference=FieldReference(
            schema_id='orientdb',
            type_name='Country',
            field_name='alpha2',
        ),
        out_edge_only=False,
    ),
    CrossSchemaEdgeDescriptor(
        edge_name='Airline_RegisteredIn',
        outbound_field_reference=FieldReference(
            schema_id='postgres',
            type_name='Airline',
            field_name='alpha2_country',
        ),
        inbound_field_reference=FieldReference(
            schema_id='orientdb',
            type_name='Country',
            field_name='alpha2',
        ),
        out_edge_only=False,
    )
]